<a href="https://colab.research.google.com/github/how-to-train-your-model/data-science-portfolio/blob/main/MSDS451_Pant_Assignment_6_Backtest_of_a_trading_strategy_on_SPY_and_determining_its_statistical_significance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 
  - Backtest of a trading strategy on SPY and determining its statistical significance

# Libraries and prep

In [1]:
!pip install sweetviz
!pip install pyfolio



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# load libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import sweetviz as sv
from sklearn.impute import SimpleImputer
from datetime import datetime
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import log_loss,accuracy_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestRegressor
import warnings
import pickle
import pyfolio as pf


/usr/local/lib/python3.8/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
# Allows for muliple output in one cell window
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Allows for viewing the complete data in a single window 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

warnings.simplefilter('ignore')


In [4]:
from google.colab import drive
drive.mount('/content/drive')
!ls -l "/content/drive/My Drive/Colab_Notebooks/MSDS-451/W6"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 5888
-rw------- 1 root root  105840 Feb 26 01:56  A6_Rough.ipynb
-rw------- 1 root root  103864 Feb 25 17:26 'MSDS 451 Assignment_05_Chan-1.ipynb'
-rw------- 1 root root  180537 Feb 26 18:10 'MSDS451_Pant_Assignment 6 - Backtest of a trading strategy on SPY and determining its statistical significance.ipynb'
-rw------- 1 root root  558120 Feb 26 17:01 'MSDS451_Pant_Assignment 6 - Backtest of a trading strategy on SPY and determining its statistical significance.ipynb.docx'
-rw------- 1 root root   24148 Feb 25 17:26  SPY.csv
-rw------- 1 root root    1372 Feb 25 18:01  test_out_seed1.csv
-rw------- 1 root root    1372 Feb 25 18:01  test_out_seed2.csv
-rw------- 1 root root    5367 Feb 25 18:01  test_seed1.csv
-rw------- 1 root root    6208 Feb 25 18:01  test_seed2.csv
-rw------- 1 root root 2473313 Feb 25 18:01  top5featRF-seed1.sav
-rw------- 1 root ro

In [5]:
w6_files = "/content/drive/My Drive/Colab_Notebooks/MSDS-451/W6"

#Q1
Backtest a naive trading strategy that buys SPY at a month’s end if RF predicts the SPX return will be positive next, and vice versa for shorts. You can download SPY data from Yahoo! FinanceLinks to an external site. or any other website. Use the first RF model from Part 4 of the Module 5: Coding Assignment. 
 - Find the annualized average return and Sharpe ratio of this strategy over the train and test sets as defined in that coding assignment (you can assume the risk-free rate to be zero when computing Sharpe ratio). 

- Why can’t we backtest a trading strategy using the SPX index directly? (Note: Yahoo! Finance data starts in 1993, so you should cut off any feature data before that.) (3 pt)

## seed 1 data

In [6]:
# Create dataframe

#train files
X_train1 = pd.read_csv(w6_files+"/train_seed1.csv")
Y_train1 = pd.read_csv(w6_files+"/train_out_seed1.csv")
X_train1.set_index('yyyymm', inplace = True)
Y_train1.set_index('yyyymm', inplace = True)

X_train1.head(2)
Y_train1.head(2)


,lty_FD,d/e,tms
yyyymm,,,
1945-12-01,-1.507835,-0.374693,0.0161
1946-01-01,-1.486065,-0.343589,0.0161


,ret
yyyymm,
1945-12-01,1.0
1946-01-01,-1.0


In [7]:
# test files
X_test1 = pd.read_csv(w6_files+"/test_seed1.csv")
Y_test1 = pd.read_csv(w6_files+"/test_out_seed1.csv")
X_test1.set_index('yyyymm', inplace = True)
Y_test1.set_index('yyyymm', inplace = True)
X_test1.head(2)
Y_test1.head()

,lty_FD,d/e,tms
yyyymm,,,
2012-06-01,-1.437708,-1.132859,0.0224
2012-07-01,-1.503595,-1.112623,0.0206


,ret
yyyymm,
2012-06-01,1.0
2012-07-01,1.0
2012-08-01,1.0
2012-09-01,-1.0
2012-10-01,1.0


In [8]:
#Load SPY data
SPY_data = pd.read_csv(w6_files+"/SPY.csv")
SPY_data.set_index('Date', inplace = True)
SPY_data.head(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-02-01,43.96875,45.12500,42.81250,44.40625,26.160336,5417600
1993-03-01,44.56250,45.84375,44.21875,45.18750,26.620569,3019200


In [9]:
# load the model from disk
loaded_model_seed1 = pickle.load(open(w6_files+"/top5featRF-seed1.sav", 'rb'))
result = loaded_model_seed1.score(X_test1, Y_test1)
print(result)

0.651685393258427


In [10]:
X_test11 = X_test1.copy(deep = True)

In [11]:
print('================ X_train1 view ================')
X_train1.shape
X_train1.head(2)
X_train1.dtypes

print('================ X_test1 view ================')
X_test1.shape
X_test1.head(2)
X_test1.dtypes

print('================ SPY_data view ================')
SPY_data.shape
SPY_data.head(2)
SPY_data.dtypes


================ X_train1 view ================


(621, 3)

,lty_FD,d/e,tms
yyyymm,,,
1945-12-01,-1.507835,-0.374693,0.0161
1946-01-01,-1.486065,-0.343589,0.0161


lty_FD    float64
d/e       float64
tms       float64
dtype: object

================ X_test1 view ================


...

lty_FD    float64
d/e       float64
tms       float64
dtype: object

================ SPY_data view ================


(323, 6)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-02-01,43.96875,45.12500,42.81250,44.40625,26.160336,5417600
1993-03-01,44.56250,45.84375,44.21875,45.18750,26.620569,3019200


Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [12]:
print('================ X_train1 date range ================')
X_train1.index
X_train1.index = pd.to_datetime(X_train1.index)
X_train1.index.dtype

print('================ X_test1 date range ================')
X_test11.index
#X_test1.index = pd.to_datetime(X_test1.index)
X_test11.index.dtype

print('================ SPY_data date range ================')
SPY_data.index
SPY_data.index = pd.to_datetime(SPY_data.index)
SPY_data.index.dtype


================ X_train1 date range ================


Index(['1945-12-01', '1946-01-01', '1946-02-01', '1946-03-01', '1946-04-01', '1946-05-01', '1946-06-01', '1946-07-01', '1946-08-01', '1946-09-01',
       ...
       '1996-11-01', '1996-12-01', '1997-01-01', '1997-02-01', '1997-03-01', '1997-04-01', '1997-05-01', '1997-06-01', '1997-07-01', '1997-08-01'], dtype='object', name='yyyymm', length=621)

dtype('<M8[ns]')

================ X_test1 date range ================


Index(['2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01', '2012-11-01', '2012-12-01', '2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01', '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01', '2013-09-01', '2013-10-01', '2013-11-01', '2013-12-01', '2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01', '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01', '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01', '2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01', '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01', '2018-03-01',
       '2018-

dtype('O')

================ SPY_data date range ================


Index(['1993-02-01', '1993-03-01', '1993-04-01', '1993-05-01', '1993-06-01', '1993-07-01', '1993-08-01', '1993-09-01', '1993-10-01', '1993-11-01',
       ...
       '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01'], dtype='object', name='Date', length=323)

dtype('<M8[ns]')

The test dataframe has values from 2021-06-01 to 2019-10-01. To keep the data consistent, we will filter the dataframes for these dates. 

In [13]:
# X_train1_fil = X_train1.loc[(X_train1['Date'] >= '2012-06-01')
#                      & (X_train1['Date'] <= '2019-10-01')]
start_date = '2012-06-01'
end_date = '2019-10-01'
# X_train1_fil = X_train1.loc[(X_train1.index >= start_date) & 
#                             (X_train1.index <= end_date)]
              
# X_test1_fil = X_test1.loc[(X_test1.index >= start_date) & 
#                             (X_test1.index <= end_date)]

SPY_data_fil = SPY_data.loc[(SPY_data.index >= start_date) & 
                            (SPY_data.index <= end_date)]

In [14]:

predictions = []
for date in X_test11.index:
    row = X_test11.loc[date]
    prediction = loaded_model_seed1.predict([[row['lty_FD'], row['d/e'], row['tms']]])
    predictions.append(prediction[0])

# add the predictions as a new column to the DataFrame
X_test11['Prediction'] = predictions

In [15]:
X_test11['Prediction'].value_counts()

 1    78
-1    11
Name: Prediction, dtype: int64

In [16]:
print('================ X_test1 view ================')
X_test11.index = pd.to_datetime(X_test1.index)
X_test11.shape
X_test11.head(2)
X_test11.dtypes

print('================ SPY_data view ================')
SPY_data_fil.shape
SPY_data_fil.head(2)
SPY_data_fil.dtypes


================ X_test1 view ================


(89, 4)

,lty_FD,d/e,tms,Prediction
yyyymm,,,,
2012-06-01,-1.437708,-1.132859,0.0224,1
2012-07-01,-1.503595,-1.112623,0.0206,1


lty_FD        float64
d/e           float64
tms           float64
Prediction      int64
dtype: object

================ SPY_data view ================


(89, 6)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-06-01,129.410004,136.270004,127.139999,136.100006,114.001549,3644886200
2012-07-01,136.479996,139.339996,132.600006,137.710007,115.947792,2864743300


Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [17]:
joined_df1 = SPY_data_fil.join(X_test11)
#joined_df = pd.merge(SPY_data_fil, X_test1_fil, on='Date', how='inner')

joined_df1.shape
joined_df1['Prediction'].value_counts()

(89, 10)

 1    78
-1    11
Name: Prediction, dtype: int64

In [18]:
joined_df1.index


DatetimeIndex(['2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01', '2012-11-01', '2012-12-01', '2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01', '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01', '2013-09-01', '2013-10-01', '2013-11-01', '2013-12-01', '2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01', '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01', '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01', '2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01', '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01', '2018-03-01',
     

In [19]:
# EDA
print("\n\n Duplicates",joined_df1.duplicated().sum())

print("\n\n Missing Values before dropping/Imputing \n",joined_df1.isna().sum())



 Duplicates 0


 Missing Values before dropping/Imputing 
 Open          0
High          0
Low           0
Close         0
Adj Close     0
Volume        0
lty_FD        0
d/e           0
tms           0
Prediction    0
dtype: int64


In [20]:
joined_df1.dropna(inplace=True)
joined_df1.shape


(89, 10)

In [21]:
#adjust for only the times the signal switches.
joined_df1 = joined_df1[joined_df1.Prediction.shift(1)!=joined_df1.Prediction]
joined_df1.shape
joined_df1.head(5)

(17, 10)

,Open,High,Low,Close,Adj Close,Volume,lty_FD,d/e,tms,Prediction
Date,,,,,,,,,,
2012-06-01,129.410004,136.270004,127.139999,136.100006,114.001549,3644886200,-1.437708,-1.132859,0.0224,1
2013-06-01,163.830002,165.990005,155.729996,160.419998,137.361328,3533321800,-1.174109,-1.005764,0.0325,-1
2013-07-01,161.259995,169.860001,160.220001,168.710007,145.224106,2330904500,-1.169297,-1.006886,0.0340,1
2013-10-01,168.139999,177.509995,164.529999,175.789993,152.056061,2901739000,-1.236763,-1.023754,0.0337,-1
2013-11-01,176.020004,181.750000,174.759995,181.000000,156.562637,1930952900,-1.185525,-1.038081,0.0354,1


In [22]:
#You don’t know when the first signal was set because you have a censored data set. 
#The first signal change you know is the 2nd one. 

joined_df1=joined_df1.iloc[1:,:]
joined_df1.shape

(16, 10)

In [23]:
joined_df1['ret']=(joined_df1['Adj Close'].shift(-1)-
                  joined_df1['Adj Close'])/joined_df1['Adj Close']*joined_df1.Prediction

In [24]:
joined_df1['ret'].head()

Date
2013-06-01   -0.057242
2013-07-01    0.047044
2013-10-01   -0.029638
2013-11-01   -0.010236
2014-01-01   -0.061517
Name: ret, dtype: float64

### Sharpe Ratio

In [25]:
perf_func = pf.timeseries.perf_stats

def get_daily_returns(intraday_returns):
    """
    This changes returns into daily returns that will work using pyfolio. Its not perfect...
    """
    cum_rets = ((intraday_returns + 1).cumprod())

    # Downsample to daily
    daily_rets = cum_rets.resample('B').last()

    # Forward fill, Percent Change, Drop NaN
    daily_rets = daily_rets.ffill().pct_change().dropna()

    return daily_rets

 

daily_rets1 = get_daily_returns(joined_df1['ret'])
pf.show_perf_stats(daily_rets1)
daily_rets1

Start date,2013-06-03
End date,2018-11-30
Total months,68
,Backtest
Annual return,8.2%
Cumulative returns,56.4%
Annual volatility,19.0%
Sharpe ratio,0.49
Calmar ratio,0.83
Stability,0.74
Max drawdown,-9.9%


Date
2013-06-03    0.0
2013-06-04    0.0
2013-06-05    0.0
2013-06-06    0.0
2013-06-07    0.0
             ... 
2018-11-26    0.0
2018-11-27    0.0
2018-11-28    0.0
2018-11-29    0.0
2018-11-30    0.0
Freq: B, Name: ret, Length: 1435, dtype: float64

In [51]:
from numpy import sqrt 

ann_avg_return = pf.timeseries.annual_return(joined_df1['ret'])
print('----- annualized average return : -------',ann_avg_return)

Sharpe_Ratio = .49
Annual_Return = 8.2
Risk_Free_Rate = 0

Monthly_Sharpe_Ratio = Sharpe_Ratio / sqrt(12)

print('\n\n----- Monthly_Sharpe_Ratio : -------',Monthly_Sharpe_Ratio)


# calculate the performance statistics for the portfolio
perf_stats = pf.timeseries.perf_stats(joined_df1['ret'])

# extract the annualized average Sharpe ratio from the performance statistics
ann_sharpe_ratio = perf_stats['Sharpe ratio'] * (252 ** 0.5)

print("Annualized Average Sharpe Ratio: {:.2f}".format(ann_sharpe_ratio))

----- annualized average return : ------- 452.015652033817


----- Monthly_Sharpe_Ratio : ------- 0.1414508159514583
Annualized Average Sharpe Ratio: 67.80


# Q2

Based on Bailey and López de Prado (2011) which you can find on the Overview Page, what is the approximate probability (to within 1 significant digit) that the “true” Sharpe ratio is greater than 0 for your backtest over the test set? (Hint: you can use mlfinlab’s Backtest Statistics module to compute that. See https://github.com/gahobbsau/research/blob/master/Chapter14_BacktestStatistics/Chapter14_BacktestStatistics.ipynbLinks to an external site. on how to use that library.) (3 pt)

## PSR

In [27]:
def probabilistic_sharpe_ratio(returns, risk_free_rate=0, sample_size=10000, alpha=0.05):
    """
    Computes the Probabilistic Sharpe Ratio (PSR) of a series of returns.
    Parameters:
        returns (array-like): An array-like object (e.g. list, numpy array) of asset returns.
        risk_free_rate (float): The risk-free rate of return (default: 0).
        sample_size (int): The number of bootstrap samples to generate (default: 10000).
        alpha (float): The significance level for computing the confidence interval (default: 0.05).
    Returns:
        float: The PSR of the input returns.
    """
    T = len(returns)
    sharpe_ratio = np.sqrt(T) * np.mean(returns - risk_free_rate) / np.std(returns - risk_free_rate)
    bootstrap_sharpe_ratios = np.empty(sample_size)
    for i in range(sample_size):
        bootstrap_returns = np.random.choice(returns, size=T, replace=True)
        bootstrap_sharpe_ratios[i] = np.sqrt(T) * np.mean(bootstrap_returns - risk_free_rate) / np.std(bootstrap_returns - risk_free_rate)
    lower_bound = np.percentile(bootstrap_sharpe_ratios, 100 * alpha / 2)
    upper_bound = np.percentile(bootstrap_sharpe_ratios, 100 * (1 - alpha / 2))
    psr = (sharpe_ratio - lower_bound) / (upper_bound - lower_bound)
    return psr

 

risk_free_rate = 0.0

In [52]:
psr = probabilistic_sharpe_ratio(daily_rets1, risk_free_rate, sample_size=1506)
print(psr)

0.7069039831481803


# Q3

Repeat Part #1 by using the RF model created in Part #6 of the coding assignment in Module 5. Does the Sharpe ratio over the test set change? (2 pt)

## seed 2 data

In [29]:
# Create dataframe

#train files
X_train2 = pd.read_csv(w6_files+"/train_seed2.csv")
Y_train2 = pd.read_csv(w6_files+"/train_out_seed2.csv")
X_train2.set_index('yyyymm', inplace = True)
Y_train2.set_index('yyyymm', inplace = True)

X_train2.head(2)
Y_train2.head(2)


,e/p,d/e,infl
yyyymm,,,
1945-12-01,-2.894991,-0.374693,0.005525
1946-01-01,-2.983423,-0.343589,0.000000


,ret
yyyymm,
1945-12-01,1.0
1946-01-01,-1.0


In [30]:
# test files
X_test2 = pd.read_csv(w6_files+"/test_seed2.csv")
Y_test2 = pd.read_csv(w6_files+"/test_out_seed2.csv")
X_test2.set_index('yyyymm', inplace = True)
Y_test2.set_index('yyyymm', inplace = True)
X_test2.head(2)
Y_test2.head()

,e/p,d/e,infl
yyyymm,,,
2012-06-01,-2.740400,-1.132859,-0.001466
2012-07-01,-2.758317,-1.112623,-0.001630


,ret
yyyymm,
2012-06-01,1.0
2012-07-01,1.0
2012-08-01,1.0
2012-09-01,-1.0
2012-10-01,1.0


In [31]:
#Load SPY data
SPY_data = pd.read_csv(w6_files+"/SPY.csv")
SPY_data.set_index('Date', inplace = True)
SPY_data.head(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-02-01,43.96875,45.12500,42.81250,44.40625,26.160336,5417600
1993-03-01,44.56250,45.84375,44.21875,45.18750,26.620569,3019200


In [32]:
# load the model from disk
loaded_model_seed2 = pickle.load(open(w6_files+"/top5featRF-seed2.sav", 'rb'))
result = loaded_model_seed2.score(X_test2, Y_test2)
print(result)

0.5617977528089888


In [33]:
X_test22 = X_test2.copy(deep = True)

In [34]:
print('================ X_train1 view ================')
X_train2.shape
X_train2.head(2)
X_train2.dtypes

print('================ X_test1 view ================')
X_test2.shape
X_test2.head(2)
X_test2.dtypes

print('================ SPY_data view ================')
SPY_data.shape
SPY_data.head(2)
SPY_data.dtypes


================ X_train1 view ================


(621, 3)

,e/p,d/e,infl
yyyymm,,,
1945-12-01,-2.894991,-0.374693,0.005525
1946-01-01,-2.983423,-0.343589,0.000000


e/p     float64
d/e     float64
infl    float64
dtype: object

================ X_test1 view ================


...

e/p     float64
d/e     float64
infl    float64
dtype: object

================ SPY_data view ================


(323, 6)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-02-01,43.96875,45.12500,42.81250,44.40625,26.160336,5417600
1993-03-01,44.56250,45.84375,44.21875,45.18750,26.620569,3019200


Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [35]:
print('================ X_train1 date range ================')
X_train2.index
X_train2.index = pd.to_datetime(X_train2.index)
X_train2.index.dtype

print('================ X_test1 date range ================')
X_test22.index
#X_test1.index = pd.to_datetime(X_test1.index)
X_test22.index.dtype

print('================ SPY_data date range ================')
SPY_data.index
SPY_data.index = pd.to_datetime(SPY_data.index)
SPY_data.index.dtype


================ X_train1 date range ================


Index(['1945-12-01', '1946-01-01', '1946-02-01', '1946-03-01', '1946-04-01', '1946-05-01', '1946-06-01', '1946-07-01', '1946-08-01', '1946-09-01',
       ...
       '1996-11-01', '1996-12-01', '1997-01-01', '1997-02-01', '1997-03-01', '1997-04-01', '1997-05-01', '1997-06-01', '1997-07-01', '1997-08-01'], dtype='object', name='yyyymm', length=621)

dtype('<M8[ns]')

================ X_test1 date range ================


Index(['2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01', '2012-11-01', '2012-12-01', '2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01', '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01', '2013-09-01', '2013-10-01', '2013-11-01', '2013-12-01', '2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01', '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01', '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01', '2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01', '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01', '2018-03-01',
       '2018-

dtype('O')

================ SPY_data date range ================


Index(['1993-02-01', '1993-03-01', '1993-04-01', '1993-05-01', '1993-06-01', '1993-07-01', '1993-08-01', '1993-09-01', '1993-10-01', '1993-11-01',
       ...
       '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01'], dtype='object', name='Date', length=323)

dtype('<M8[ns]')

The test dataframe has values from 2021-06-01 to 2019-10-01. To keep the data consistent, we will filter the dataframes for these dates. 

In [36]:
# X_train1_fil = X_train1.loc[(X_train1['Date'] >= '2012-06-01')
#                      & (X_train1['Date'] <= '2019-10-01')]
start_date = '2012-06-01'
end_date = '2019-10-01'
# X_train1_fil = X_train1.loc[(X_train1.index >= start_date) & 
#                             (X_train1.index <= end_date)]
              
# X_test1_fil = X_test1.loc[(X_test1.index >= start_date) & 
#                             (X_test1.index <= end_date)]

SPY_data_fil = SPY_data.loc[(SPY_data.index >= start_date) & 
                            (SPY_data.index <= end_date)]

In [37]:

predictions = []
for date in X_test22.index:
    row = X_test22.loc[date]
    prediction = loaded_model_seed2.predict([[row['e/p'], row['d/e'], row['infl']]])
    predictions.append(prediction[0])

# add the predictions as a new column to the DataFrame
X_test22['Prediction'] = predictions

In [38]:
X_test22['Prediction'].value_counts()

 1    48
-1    41
Name: Prediction, dtype: int64

In [39]:
print('================ X_test1 view ================')
X_test22.index = pd.to_datetime(X_test2.index)
X_test22.shape
X_test22.head(2)
X_test22.dtypes

print('================ SPY_data view ================')
SPY_data_fil.shape
SPY_data_fil.head(2)
SPY_data_fil.dtypes


================ X_test1 view ================


(89, 4)

,e/p,d/e,infl,Prediction
yyyymm,,,,
2012-06-01,-2.740400,-1.132859,-0.001466,1
2012-07-01,-2.758317,-1.112623,-0.001630,1


e/p           float64
d/e           float64
infl          float64
Prediction      int64
dtype: object

================ SPY_data view ================


(89, 6)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-06-01,129.410004,136.270004,127.139999,136.100006,114.001549,3644886200
2012-07-01,136.479996,139.339996,132.600006,137.710007,115.947792,2864743300


Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [40]:
joined_df2 = SPY_data_fil.join(X_test22)
#joined_df = pd.merge(SPY_data_fil, X_test1_fil, on='Date', how='inner')

joined_df2.shape
joined_df2['Prediction'].value_counts()

(89, 10)

 1    48
-1    41
Name: Prediction, dtype: int64

In [41]:
joined_df2.index


DatetimeIndex(['2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01', '2012-11-01', '2012-12-01', '2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01', '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01', '2013-09-01', '2013-10-01', '2013-11-01', '2013-12-01', '2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01', '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01', '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01', '2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01', '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01', '2018-03-01',
     

In [42]:
# EDA
print("\n\n Duplicates",joined_df2.duplicated().sum())

print("\n\n Missing Values before dropping/Imputing \n",joined_df2.isna().sum())



 Duplicates 0


 Missing Values before dropping/Imputing 
 Open          0
High          0
Low           0
Close         0
Adj Close     0
Volume        0
e/p           0
d/e           0
infl          0
Prediction    0
dtype: int64


In [43]:
joined_df2.dropna(inplace=True)
joined_df2.shape


(89, 10)

In [44]:
#adjust for only the times the signal switches.
joined_df2 = joined_df2[joined_df2.Prediction.shift(1)!=joined_df2.Prediction]
joined_df2.shape
joined_df2.sort_index().head(5)

(22, 10)

,Open,High,Low,Close,Adj Close,Volume,e/p,d/e,infl,Prediction
Date,,,,,,,,,,
2012-06-01,129.410004,136.270004,127.139999,136.100006,114.001549,3644886200,-2.740400,-1.132859,-0.001466,1
2013-02-01,150.649994,153.279999,148.729996,151.610001,129.242783,2581459300,-2.853571,-1.009180,0.008190,-1
2013-03-01,151.089996,156.850006,150.410004,156.669998,133.556305,2330972300,-2.884393,-1.004694,0.002615,1
2015-04-01,206.389999,212.479996,204.509995,208.460007,185.661713,2036685200,-3.059810,-0.866503,0.002033,-1
2015-05-01,209.399994,213.779999,206.759995,211.139999,188.048615,1892519900,-3.085149,-0.844046,0.005097,1


In [45]:
#You don’t know when the first signal was set because you have a censored data set. 
#The first signal change you know is the 2nd one. 

joined_df2=joined_df2.iloc[1:,:]

In [46]:
joined_df2.shape
joined_df2.sort_index().head(5)

(21, 10)

,Open,High,Low,Close,Adj Close,Volume,e/p,d/e,infl,Prediction
Date,,,,,,,,,,
2013-02-01,150.649994,153.279999,148.729996,151.610001,129.242783,2581459300,-2.853571,-1.009180,0.008190,-1
2013-03-01,151.089996,156.850006,150.410004,156.669998,133.556305,2330972300,-2.884393,-1.004694,0.002615,1
2015-04-01,206.389999,212.479996,204.509995,208.460007,185.661713,2036685200,-3.059810,-0.866503,0.002033,-1
2015-05-01,209.399994,213.779999,206.759995,211.139999,188.048615,1892519900,-3.085149,-0.844046,0.005097,1
2015-07-01,207.729996,213.179993,204.110001,210.500000,188.390533,2452758000,-3.113629,-0.800265,0.000067,-1


In [47]:
joined_df2['ret']=(joined_df2['Adj Close'].shift(-1)-
                  joined_df2['Adj Close'])/joined_df2['Adj Close']*joined_df2.Prediction

### Sharpe Ratio

In [48]:
perf_func = pf.timeseries.perf_stats

def get_daily_returns(intraday_returns):
    """
    This changes returns into daily returns that will work using pyfolio. Its not perfect...
    """
    cum_rets = ((intraday_returns + 1).cumprod())

    # Downsample to daily
    daily_rets = cum_rets.resample('B').last()

    # Forward fill, Percent Change, Drop NaN
    daily_rets = daily_rets.ffill().pct_change().dropna()

    return daily_rets

 

daily_rets2 = get_daily_returns(joined_df2['ret'])
pf.show_perf_stats(daily_rets2)

Start date,2013-02-04
End date,2019-07-01
Total months,79
,Backtest
Annual return,2.9%
Cumulative returns,20.5%
Annual volatility,17.7%
Sharpe ratio,0.24
Calmar ratio,0.11
Stability,0.57
Max drawdown,-27.0%


In [49]:
from numpy import sqrt 

ann_avg_return = pf.timeseries.annual_return(joined_df2['ret'])
print('----- annualized average return : -------',ann_avg_return)

Sharpe_Ratio = .24
Annual_Return = 2.9
Risk_Free_Rate = 0

Monthly_Sharpe_Ratio = Sharpe_Ratio / sqrt(12)

print('\n\n----- Monthly_Sharpe_Ratio : -------',Monthly_Sharpe_Ratio)


# calculate the performance statistics for the portfolio
perf_stats = pf.timeseries.perf_stats(joined_df2['ret'])

# extract the annualized average Sharpe ratio from the performance statistics
ann_sharpe_ratio = perf_stats['Sharpe ratio'] * (252 ** 0.5)

print("Annualized Average Sharpe Ratio: {:.2f}".format(ann_sharpe_ratio))

----- annualized average return : ------- 5.243100874480894


----- Monthly_Sharpe_Ratio : ------- 0.06928203230275509
Annualized Average Sharpe Ratio: 29.51


## PSR

In [50]:
psr = probabilistic_sharpe_ratio(daily_rets2, risk_free_rate, sample_size=1506)
print(psr)

0.6225196069140153


## Q4

Instead of using the theoretical results of the paper cited in Part 2, what would be an empirical way to find out the probability that your “true” Sharpe ratio will be positive? (2 pt)

We could use a few statistical measures to find Sharpe ratio in an empirical way

Correlation: The Sharpe ratio assumes that returns are independent and identically distributed (i.i.d.), meaning that there is no correlation between returns. If returns are correlated, the Sharpe ratio may overestimate the risk-adjusted return of the investment or trading strategy.

Standard deviation: is a measure of the volatility of the returns of a trading strategy. The Sharpe ratio compares the average excess return of the strategy to the standard deviation of the excess returns. Therefore, the higher the standard deviation of returns, the higher the denominator of the Sharpe ratio, which would typically result in a lower Sharpe ratio. Conversely, if the standard deviation of returns is lower, the denominator of the Sharpe ratio is lower, which would typically result in a higher Sharpe ratio.

Skewness: measures the degree of asymmetry in the distribution of returns. Positive skewness indicates that the distribution of returns is skewed to the right, meaning that there are more positive returns than negative returns. In this case, the average excess return of the strategy would be positively biased, which would typically result in a higher Sharpe ratio. 